In [ ]:
# import all the libraries needed to run this code
import pandas as pd
import numpy as np
import os, re, glob
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit

%matplotlib inline

# File Sources

In [ ]:
pre_rxn_mix = "./in situ WAXS/Single-Crystalline TAPB-DMPDA - Standard Conditions/Pre Rxn Mix/"
rxn_folder =  "./in situ WAXS/Single-Crystalline TAPB-DMPDA - Standard Conditions/Raw Data/an-3-17-1-try2/"
avg_save_loc = "./in situ WAXS/Single-Crystalline TAPB-DMPDA - Standard Conditions/Background-Subtracted Data/"
# change folder paths as needed

# Preprocessing of Data

In [ ]:
# background_dfs is an array that stores respective averaged backgrounds for each pre-rxn mix
background_dfs = []

directories = os.listdir(pre_rxn_mix)

for directory in directories:
    if ".DS_Store" not in directory:
        all_files = glob.glob(os.path.join(f"{pre_rxn_mix+directory}/plot_files/","*.txt"))
        df = pd.concat((pd.read_csv(f, delimiter="\t", skiprows=48) for f in all_files)) 
        # first 48 rows are metadata in all pre-rxn mix files
        df = df.rename(columns={"I (1/cm)": "I (1/cm)_bg"})
        background_dfs.append(df.groupby(["q (1/A)"])["I (1/cm)_bg"].mean().reset_index())

In [ ]:
# describe the statistical properties of each column
background_dfs[0].describe()

In [ ]:
# plot a given position in the background_dfs array on a loglog scale
background_dfs[0].plot(x="q (1/A)", y="I (1/cm)_bg", loglog = True)

In [ ]:
# ensure data is stored properly in the array background_dfs
print(background_dfs)

In [ ]:
# get each rxn folder individually to subtract averaged background
all_rxn_files = glob.glob(os.path.join(f"{rxn_folder}plot_files/","*.txt"))

for f in all_rxn_files:
    print (f)
    df_rxn = pd.read_csv(f, delimiter="\t", skiprows=48)
    # print (df_rxn.columns)
    # print (f)
    
    # try/except block accounts for some data files having 48 rows of metadata 
    # and others having 49 or 50 rows of metadata
    try:
        df_rxn = pd.read_csv(f, delimiter="\t", skiprows=48)
        test_df = df_rxn.join(background_dfs[0].set_index("q (1/A)"), on="q (1/A)", how="left")
    except:
        df_rxn = pd.read_csv(f, delimiter="\t", skiprows=49)
        test_df = df_rxn.join(background_dfs[0].set_index("q (1/A)"), on="q (1/A)", how="left")
        try:
            df_rxn = pd.read_csv(f, delimiter="\t", skiprows=50)
            test_df = df_rxn.join(background_dfs[0].set_index("q (1/A)"), on="q (1/A)", how="left")
        except:
            pass
    test_df["I (1/cm) sub"] = test_df["I (1/cm)"] - test_df["I (1/cm)_bg"]
    filename_without_extension = os.path.splitext(os.path.basename(f))[0]
    csv_filepath = os.path.join(avg_save_loc, "plot_files", f"{filename_without_extension}.csv")
    test_df.to_csv(csv_filepath, index=False)  

In [ ]:
test_df.describe()

In [ ]:
test_df.head()

# Import and Plot XRD Data

In [ ]:
# import in situ SAXs/WAXS data into the array comp_data
dirarray1 = os.scandir(avg_save_loc)
comp_data = []

for jj, entry in enumerate(dirarray1):
    if entry.is_dir():
        a = os.path.join(avg_save_loc, entry.name)
        dirarray = os.scandir(a)
        # Get files to import
        files, number = [], []
        for iii, file_entry in enumerate(dirarray):
            if not file_entry.is_dir():
                if re.search('\.csv$', file_entry.name):
                    if re.search('hs102', file_entry.name):
                        files.append(file_entry.name)
                        temp1 = re.search('hs102', file_entry.name).start()
                        base = file_entry.name[:temp1]
                        number.append(file_entry.name[temp1+6:temp1+14])

        n_samples = len(number)

        # import all the files
        sa = []
        for ii in range(n_samples):
            WAXS_path = os.path.join(a, base + 'hs102_' + number[ii] + '.csv')
            MAXS_path = os.path.join(a, base + 'hs103_' + number[ii] + '.csv')
            SAXS_path = os.path.join(a, base + 'hs104_' + number[ii] + '.csv')
            WAXS_df = pd.read_csv(WAXS_path)
            MAXS_df = pd.read_csv(MAXS_path)
            SAXS_df = pd.read_csv(SAXS_path)
            print (MAXS_path)
            
            # append the I and Q values to a structure
            sa.append({
                'WAXSQ': WAXS_df["q (1/A)"],
                'WAXSData': WAXS_df["I (1/cm) sub"],
                'MAXSQ': MAXS_df["q (1/A)"],
                'MAXSData': MAXS_df["I (1/cm) sub"],
                'SAXSQ': SAXS_df["q (1/A)"],
                'SAXSData': SAXS_df["I (1/cm) sub"],
                'num': number[ii]
            })

        comp_data.append({'st': sa, 'name': entry.name})

In [ ]:
# plotting data
import matplotlib.colors as mcolors
n = 256 # change as needed
color_indices = np.sqrt(np.linspace(0, 1, n))
colors = plt.cm.rainbow(color_indices)
cmap = mcolors.ListedColormap(colors)

ii = 0

# define Data Range Here
beg_SAXS = 0
end_SAXS = 1070

beg_MAXS = 0
end_MAXS = 486

beg_WAXS = 0
end_WAXS = 1002


# plot q against I for a given range of data in specified intervals
for iii in range(10, n, 10):
    
    # plots for SAXS region
    q_SAXS = comp_data[ii]['st'][iii]['SAXSQ'][beg_SAXS:end_SAXS]

    data_SAXS = comp_data[ii]['st'][iii]['SAXSData'][beg_SAXS:end_SAXS]
    # data_norm_SAXS = (data_SAXS - np.min(data_SAXS)) / (np.max(data_SAXS) - np.min(data_SAXS))

    plt.loglog((q_SAXS), (data_SAXS+iii/100), color=colors[iii])

    # plots for MAXS region
    q_MAXS = comp_data[ii]['st'][iii]['MAXSQ'][beg_MAXS:end_MAXS]

    data_MAXS = comp_data[ii]['st'][iii]['MAXSData'][beg_MAXS:end_MAXS]
    data_norm_MAXS = (data_MAXS - np.min(data_MAXS)) / (np.max(data_MAXS) - np.min(data_MAXS))

    plt.loglog((q_MAXS), (data_norm_MAXS+iii/100), color=colors[iii])

    # plots for WAXS region
    q_WAXS = comp_data[ii]['st'][iii]['WAXSQ'][beg_WAXS:end_WAXS]

    data_WAXS = comp_data[ii]['st'][iii]['WAXSData'][beg_WAXS:end_WAXS]
    data_norm_WAXS = (data_WAXS - np.min(data_WAXS)) / (np.max(data_WAXS) - np.min(data_WAXS))
    plt.loglog((q_WAXS), (data_norm_WAXS+iii/100), color=colors[iii])

plt.xlabel('q (Å$^{-1}$)', fontsize=24, fontname='Arial')
plt.ylabel('Intensity (a.u.)', fontsize=24, fontname='Arial')
plt.xticks(fontsize=20, fontname='Arial')
plt.yticks(fontsize=20, fontname='Arial')
plt.tick_params(direction='out', length=6, width=2)
ax = plt.gca()
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
plt.savefig(f"{base}10_n_10_plot_full.png", transparent=True, dpi=600)
plt.show()

## Generate Plot for SAXS region

In [ ]:
for iii in range(10, n, 10): # change as needed
    # plots for SAXS region
    q_SAXS = comp_data[ii]['st'][iii]['SAXSQ'][beg_SAXS:end_SAXS]

    data_SAXS = comp_data[ii]['st'][iii]['SAXSData'][beg_SAXS:end_SAXS]
    data_norm_SAXS = (data_SAXS - np.min(data_SAXS)) / (np.max(data_SAXS) - np.min(data_SAXS))

    plt.loglog((q_SAXS), (data_SAXS + iii / 100), color=colors[iii])
    

plt.xlabel('q (Å$^{-1}$)', fontsize=24, fontname='Arial')
plt.ylabel('Intensity (a.u.)', fontsize=24, fontname='Arial')
plt.xticks(fontsize=20, fontname='Arial')
plt.yticks(fontsize=20, fontname='Arial')
plt.yticks([])
plt.tick_params(axis='x', which='both', direction='out', length=6, width=2, labelsize=20, pad=5)
ax = plt.gca()
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
plt.savefig(f"{base}10_n_10_plot_SAXS.png", transparent=True, dpi=600) # change file name as needed
plt.show()

## Generate Plot for MAXS Region

In [ ]:
for iii in range(1, n, 10): # change as needed 
    # plots for MAXS region
    q_MAXS = comp_data[ii]['st'][iii]['MAXSQ'][beg_MAXS:end_MAXS]

    data_MAXS = comp_data[ii]['st'][iii]['MAXSData'][beg_MAXS:end_MAXS]
    data_norm_MAXS = (data_MAXS - np.min(data_MAXS)) / (np.max(data_MAXS) - np.min(data_MAXS))
    plt.plot(q_MAXS, data_norm_MAXS + iii / 100, color=colors[iii])
   
plt.xlabel('q (Å$^{-1}$)', fontsize=24, fontname='Arial')
plt.ylabel('Intensity (a.u.)', fontsize=24, fontname='Arial')
plt.xticks(fontsize=20, fontname='Arial')
plt.yticks(fontsize=20, fontname='Arial')
plt.yticks([])
plt.tick_params(axis='x', which='both', direction='out', length=6, width=2, labelsize=20, pad=5)
ax = plt.gca()
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
plt.savefig(f"{base}1_n_10_plot_MAXS.png", transparent=True, dpi=600) # change file name as needed
plt.show()

## Generate Plot for WAXS Region

In [ ]:
for iii in range(1, n, 10): # change as needed
    
    # plot for WAXS region
    q_WAXS = comp_data[ii]['st'][iii]['WAXSQ'][beg_WAXS:end_WAXS]

    data_WAXS = comp_data[ii]['st'][iii]['WAXSData'][beg_WAXS:end_WAXS]
    data_norm_WAXS = (data_WAXS - np.min(data_WAXS)) / (np.max(data_WAXS) - np.min(data_WAXS))

    plt.plot(q_WAXS, data_norm_WAXS + iii / 100, color=colors[iii])

plt.xlabel('q (Å$^{-1}$)', fontsize=24, fontname='Arial', labelpad=5)
plt.ylabel('Intensity (a.u.)', fontsize=24, fontname='Arial', labelpad=5)
plt.xticks(fontsize=20, fontname='Arial')
plt.yticks(fontsize=20, fontname='Arial')
plt.yticks([])
plt.tick_params(axis='x', which='both', direction='in', length=6, width=2, labelsize=20, pad=5)
ax = plt.gca()
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
plt.savefig(f"{base}1_n_10_plot_WAXS.png", transparent=True, dpi=600) # change file name as needed
plt.show()

# Lorentzian Fit 

In [ ]:
# define the function "lorenzFit"
def lorentzFit(x, w, x0, A, c):
        return A / np.pi * (w / 2) / ((x - x0) ** 2 + (w / 2) ** 2) + c

In [ ]:
# get normalized <100> FWHM

# define start and end indices for Lorentzian fit of MAXS data
lorentzian_start = 34
lorentzian_end = 55
ii = 0

# save FWHM values into this vector
fwhm_values = []

# loop through data in specified range
for iii in range(0, 500):
    # calculate q and theta2
    q = comp_data[ii]['st'][iii]['MAXSQ'][lorentzian_start:lorentzian_end]
    theta2 = np.arcsin(q * 0.73 / (4 * np.pi)) * 2
    
    # normalize background-subtracted intensity
    bkgsubI = comp_data[ii]['st'][iii]['MAXSData'][lorentzian_start:lorentzian_end]
    normalized_intensity = (bkgsubI - np.min(bkgsubI)) / (np.max(bkgsubI) - np.min(bkgsubI))
    
    # define initial guesses and bounds for curve fitting
    p0 = [0.0002, 0.023, 1, 0]
    bounds = ([0.000003, -0.0002, -1, -0.1], [0.2, 0.03, 1000, 0.1])

    # perform the curve fitting
    popt, _ = curve_fit(lorentzFit, theta2, normalized_intensity, p0=p0, bounds=bounds)
    w_fit, _, _, _ = popt  # Extract the FWHM value

    # append FWHM values to the list
    fwhm_values.append(w_fit)

# convert list of FWHM values to NumPy array for easier manipulation
fwhm_values = np.array(fwhm_values)

# find minimum and maximum FWHM values
min_fwhm = np.min(fwhm_values)
max_fwhm = np.max(fwhm_values)

# normalize the FWHM values using the specified formula
normalized_fwhm_values = (fwhm_values - min_fwhm) / (max_fwhm - min_fwhm)

In [ ]:
#get crystalline domain sizes using Scherrer equation

# define start and end indices for Lorentzian fit of MAXS data
lorentzian_start = 34
lorentzian_end = 55
ii = 0

# save data into this vector
transformed_fit = np.zeros(len(range(0, 500, 1)))
# print (comp_data[ii])

# loop through range
for jjj, iii in enumerate(range(0, 500, 1)):
    lorentz_fit = []
    # print (iii)
    
    # calculate theta
    q = comp_data[ii]['st'][iii]['MAXSQ'][lorentzian_start:lorentzian_end]
    theta2 = np.arcsin(q * 0.73 / (4 * np.pi)) * 2
    
    # normalize background
    bkgsubI = comp_data[ii]['st'][iii]['MAXSData'][lorentzian_start:lorentzian_end]
    normalized = (bkgsubI - np.min(bkgsubI)) / (np.max(bkgsubI) - np.min(bkgsubI))
    #print (bkgsubI)
    
    # set boundary conditions
    lorentz_width = [0.0002, 0.000003, 0.2]
    lorentz_height = [1, -1, 1000]
    lorentz_position = [0.023, -0.0002, 0.03]
    offset = [0, -0.1, 0.1]
    
    # give inputs to function
    lb = [lorentz_width[1], lorentz_position[1], lorentz_height[1], offset[1]]
    ub = [lorentz_width[2], lorentz_position[2], lorentz_height[2], offset[2]]
    a0 = [lorentz_width[0], lorentz_position[0], lorentz_height[0], offset[0]]
    
    # define initial guesses and bounds for curve fitting
    p0 = [0.0002, 0.023, 1, 0]  
    bounds = ([0.000003, -0.0002, -1, -0.1], [0.2, 0.03, 1000, 0.1]) 

    # print (normalized)
    
    # perform the curve fitting
    popt, _ = curve_fit(lorentzFit, theta2, normalized, p0=p0, bounds=bounds)
    w_fit, x0_fit, A_fit, c_fit = popt

    t = 0.9 * 0.73 / w_fit / np.cos(x0_fit / 2) # K value is 0.9 based on Scherrer equation

    # save values
    transformed_fit[jjj] = t

In [ ]:
# plot crystalline domain size as a function of time
frame_numbers = np.arange(0, 500, 1)  # generate an array of frame numbers
time_in_minutes = frame_numbers * 5 / 60  # convert frame numbers to time in minutes

plt.xlim(left=0, right=max(time_in_minutes))
plt.plot(time_in_minutes, transformed_fit, color='#D7191C')  
plt.xlabel('Time (min)', fontsize=24, fontname='Arial', labelpad=5)  
plt.ylabel('Crystalline Domain Size (Å)', fontsize=24, fontname='Arial', labelpad=5) 
plt.ylim(500, 620) # adjust this range as needed
y_ticks = np.arange(520, 610, 20)  # adjust this range as needed
plt.yticks(y_ticks)
plt.tick_params(axis='both', which='both', direction='out', length=6, width=2, labelsize=20, pad=5)  
ax = plt.gca()
ax.spines['top'].set_linewidth(2)
ax.spines['right'].set_linewidth(2)
ax.spines['left'].set_linewidth(2)
ax.spines['bottom'].set_linewidth(2)
plt.tight_layout()
plt.savefig(f"{base}Domain_Size_plot_WAXS.png", transparent=True, dpi=600) # change file name as needed
plt.show()